In [1]:
import pandas as pd
import numpy as np
import gc

In [2]:
df_events = pd.read_csv('./vi_data/events_train.csv')
df_purchases = pd.read_csv('./vi_data/purchases_train.csv')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Nápady

https://heartbeat.fritz.ai/recommender-systems-with-python-part-i-content-based-filtering-5df4940bd831
https://towardsdatascience.com/prototyping-a-recommender-system-step-by-step-part-1-knn-item-based-collaborative-filtering-637969614ea
https://www.youtube.com/watch?v=Ty3Ui_mQtxY

# Prehľad dát

In [3]:
df_events.head()

,customer_id,timestamp,event_type,product_id,title,category_name,price
0,204685f9-00f0-4a36-bab1-e0b74673c3b1,2019-07-01 00:00:01.101977 UTC,view_item,f130c542-d793-41d0-88f6-d1c52b4fe8f0,Michael Kors Nappa Leather Racer Jacket,NaN,747.49
1,552445a5-f41d-4b25-b479-4e776d9bee61,2019-07-01 00:00:01.901589 UTC,view_item,29db8f57-49bd-4108-a0bb-0d245278e999,Puma Woven Shorts Girls,NaN,11.92
2,fac01f48-a03a-41f2-acd5-b5f4e407d9c2,2019-07-01 00:00:08.954928 UTC,view_item,a90bddc4-8eb7-4c99-aa4f-2f2377c246a2,Trendyol Multi-colored Flywheel Detailed Swimwear,NaN,23.85
3,552445a5-f41d-4b25-b479-4e776d9bee61,2019-07-01 00:00:09.721589 UTC,view_item,600268e1-a2ba-4b1b-ab57-58c577412af0,Puma Woven Shorts Girls,NaN,11.92
4,bc59a341-040f-41ea-8f33-cc4a02c2d85d,2019-07-01 00:00:13.745257 UTC,view_item,b74e0b38-1efe-4d86-9718-952cc2580212,Women's dress ROXY NEW LEASEOFLIFE J KTDR,NaN,40.00


In [4]:
df_purchases.head()

,customer_id,timestamp,event_type,product_id,title,category_name,price
0,0d1b7397-7d3c-44c0-9efc-d38bf197828b,2019-07-01 00:05:54.308966 UTC,purchase_item,954f9f2c-d3ca-4236-ac9a-4ea7bcf09305,Trendyol Leopard print Bikini bottom,NaN,NaN
1,81ccae7e-e496-4997-a289-4669bf53f33e,2019-07-01 00:20:03.404186 UTC,purchase_item,75b281e5-8a16-42cb-9ae0-9a98db7a2c40,No Fear Reflect Shorts Mens,NaN,NaN
2,50777c55-8dd6-4309-a5ca-26e66c8a8279,2019-07-01 00:34:35.989935 UTC,purchase_item,0112dec8-47f5-4c2c-9109-571e2dbb6345,Trendyol Multicolor camisole Blouse,NaN,NaN
3,50777c55-8dd6-4309-a5ca-26e66c8a8279,2019-07-01 00:34:35.991935 UTC,purchase_item,0fa25a2d-2aa1-4397-82f1-5a64f3b1272d,Trendyol White High Waist Skinny Jeans,NaN,NaN
4,647d269f-b18f-4558-8653-93369d862ec9,2019-07-01 00:52:53.083698 UTC,purchase_item,81c01216-55a9-4588-a722-bccf0bf35fd5,Firetrap Lazer Polo Shirt,NaN,NaN


## Informácie o dátach

In [5]:
df_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14614385 entries, 0 to 14614384
Data columns (total 7 columns):
customer_id      object
timestamp        object
event_type       object
product_id       object
title            object
category_name    object
price            float64
dtypes: float64(1), object(6)
memory usage: 780.5+ MB


In [6]:
df_purchases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188712 entries, 0 to 188711
Data columns (total 7 columns):
customer_id      188712 non-null object
timestamp        188712 non-null object
event_type       188712 non-null object
product_id       188712 non-null object
title            188684 non-null object
category_name    143814 non-null object
price            143826 non-null float64
dtypes: float64(1), object(6)
memory usage: 10.1+ MB


# Predspracovanie dát

In [7]:
from sklearn import preprocessing

In [8]:
# spojenie dát
df = pd.concat([df_events, df_purchases])

del [[df_events, df_purchases]]
gc.collect()

0

In [9]:
df.sort_values(by=['timestamp'], inplace=True, ascending=True)

In [10]:
df.drop(['price'], axis=1, inplace=True)

In [11]:
df['event_type'] = df['event_type'].replace({ 'view_item': 1, 'purchase_item': 2 })

## Zakódovanie idčiek

In [12]:
from sklearn import preprocessing

In [13]:
le_customer = preprocessing.LabelEncoder()
le_product = preprocessing.LabelEncoder()

df['customer_id'] = le_customer.fit_transform(df['customer_id'])
df['product_id'] = le_product.fit_transform(df['product_id'])

## Rozdelenie na trénovaciu a testovaciu zložku

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
df_train, df_test = train_test_split(df, test_size=0.20, shuffle=False)

In [16]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11842477 entries, 0 to 11689792
Data columns (total 6 columns):
customer_id      int64
timestamp        object
event_type       int64
product_id       int64
title            object
category_name    object
dtypes: int64(3), object(3)
memory usage: 632.5+ MB


In [17]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2960620 entries, 11689793 to 14614384
Data columns (total 6 columns):
customer_id      int64
timestamp        object
event_type       int64
product_id       int64
title            object
category_name    object
dtypes: int64(3), object(3)
memory usage: 158.1+ MB


### Prehľad trénovacích a testovacích dát

In [18]:
df_test.head()

,customer_id,timestamp,event_type,product_id,title,category_name
11689793,1218557,2019-08-21 07:57:53.361388 UTC,1,26125,Trendyol Multi-colored Plisse Skirt,Women>Women's Clothing>Skirts>Midi skirts
11689794,545607,2019-08-21 07:57:53.372592 UTC,1,54241,"Trendyol He's The Only Color We Block, Swimsuit",Women>Women's Clothing>Swimmsuits & bikins>One...
11689795,881435,2019-08-21 07:57:53.532665 UTC,1,54376,Women's jacket NORTHFINDER EMMALINE,Women>Women's Clothing>Jackets & Coats>Quilted...
11689796,262051,2019-08-21 07:57:53.700887 UTC,1,27195,Men's t-shirt KILPI AREC-M,Men>Men Clothing>T-Shirts & Polo shirts>Short ...
11689797,2064097,2019-08-21 07:57:53.86051 UTC,1,104807,Made Of Emotion Woman's Dress M417 Navy Blue,Women>Women's Clothing>Dresses>Mini dress


In [19]:
df_train.head()

,customer_id,timestamp,event_type,product_id,title,category_name
0,279709,2019-07-01 00:00:01.101977 UTC,1,116557,Michael Kors Nappa Leather Racer Jacket,NaN
1,735927,2019-07-01 00:00:01.901589 UTC,1,20221,Puma Woven Shorts Girls,NaN
2,2166327,2019-07-01 00:00:08.954928 UTC,1,81561,Trendyol Multi-colored Flywheel Detailed Swimwear,NaN
3,735927,2019-07-01 00:00:09.721589 UTC,1,46573,Puma Woven Shorts Girls,NaN
4,1628326,2019-07-01 00:00:13.745257 UTC,1,88433,Women's dress ROXY NEW LEASEOFLIFE J KTDR,NaN


In [20]:
df_train

,customer_id,timestamp,event_type,product_id,title,category_name
0,279709,2019-07-01 00:00:01.101977 UTC,1,116557,Michael Kors Nappa Leather Racer Jacket,NaN
1,735927,2019-07-01 00:00:01.901589 UTC,1,20221,Puma Woven Shorts Girls,NaN
2,2166327,2019-07-01 00:00:08.954928 UTC,1,81561,Trendyol Multi-colored Flywheel Detailed Swimwear,NaN
3,735927,2019-07-01 00:00:09.721589 UTC,1,46573,Puma Woven Shorts Girls,NaN
4,1628326,2019-07-01 00:00:13.745257 UTC,1,88433,Women's dress ROXY NEW LEASEOFLIFE J KTDR,NaN
...,...,...,...,...,...,...
11689788,1026488,2019-08-21 07:57:52.709547 UTC,1,78066,Karrimor 3 in 1 Weathertite Jacket Ladies,Sports>Outdoor>Outdoor Clothing>Women's Outdoo...
11689789,2096704,2019-08-21 07:57:52.718063 UTC,1,110596,Lee Cooper Jimin Juniors Trainers,Women>Women's Footwear>Sneakers>Low sneakers
11689790,340302,2019-08-21 07:57:53.004127 UTC,1,93735,Trendyol Cinnamon Belted Dress,Women>Women's Clothing>Dresses>Midi dress
11689791,132705,2019-08-21 07:57:53.080232 UTC,1,70957,Troll LADY'S JACKET,Women>Women's Clothing>Jackets & Coats>Quilted...


# Tvorba sparse matrix

robím podla tohoto
https://towardsdatascience.com/prototyping-a-recommender-system-step-by-step-part-1-knn-item-based-collaborative-filtering-637969614ea

Ktorú sparse_matrix použiť:
https://cmdlinetips.com/2018/03/sparse-matrices-in-python-with-scipy/

In [21]:
from scipy.sparse import coo_matrix

In [22]:
# tvorba sparse matrix
mat_coo = coo_matrix(
    (
        np.array(df_train['event_type']), 
        (
            np.array(df_train['product_id']), 
            np.array(df_train['customer_id'])
        )
    )
)

In [23]:
mat_csr = mat_coo.tocsr(copy=False)

In [24]:
mat_csr.shape

(123837, 2211386)

# Nájdenie najpopulárnejších produktov

In [25]:
summed_event_type = df_train[['product_id', 'event_type']].groupby(['product_id']).sum().sort_values(by=['event_type'], ascending=False)

# Odporúčanie

In [64]:
import operator
from sklearn.neighbors import NearestNeighbors

In [65]:
# vytvorenie inštancie KNN
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10, n_jobs=-1)

# natrénovanie na trénovacích dátach
model_knn.fit(mat_csr)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=-1, n_neighbors=10, p=2,
                 radius=1.0)

In [83]:
# funkcia, ktorá vráti odporúčania pre daného customera
def recommend(model_knn, customer_id, n_recos): 
    
    # 1. produkty, s ktorými interagoval customer v teste
    interacted_products_test = list(dict.fromkeys(df_test[df_test['customer_id'] == customer_id]['product_id'].values))
    
    # 2. Nájdi všetky produkty, ktoré si zákazník pozrel/kúpil v train
    interacted_products_train = list(dict.fromkeys(df_train[df_train['customer_id'] == customer_id]['product_id'].values))
    
    # 3. Ak nema ziadne produkty v train, tak daj generické odporúčania a skonči
    if len(interacted_products_train) == 0:
        most_popular_products = list(summed_event_type.head(n_recos).reset_index()['product_id'])
        precision = calc_precision(
            most_popular_products,
            list(dict.fromkeys(interacted_products_test)) # odstrániť duplikáty z pola produktov pozretých v testovacej sade
        )
        return [], most_popular_products, precision

        
    # 4. Generovanie odporúčaní pomocou KNN
    # n_recos+1, pretože vždy vráti aj seba
    distances2D, indices2D = model_knn.kneighbors(mat_csr[interacted_products_train], n_neighbors=n_recos+1)
    
    
    # 5. Vytvorenie mapy s najmenšími vzdialenostami pre každý produkt
    index_distance_map = {}
    for distances, indices in zip(distances2D, indices2D):
        for distance, index in zip(distances[1:], indices[1:]): 
            if index in index_distance_map and index_distance_map[index] < distance:
                continue
            index_distance_map[index] = distance
            

    # 6. nájdi najlepších n (n_recos) výsledkov
    indices = []
    distances = []
    for _ in range(n_recos):
        index = min(index_distance_map.items(), key=operator.itemgetter(1))[0]
        indices.append(index)
        distances.append(index_distance_map[index])
        del index_distance_map[index]
    
        
    # 7. Výpočet precision
    precision = calc_precision(
        indices,
        list(dict.fromkeys(interacted_products_test)) # odstrániť duplikáty z pola produktov pozretých v testovacej sade
    )
    
    # 8. Výpočet recall
    recall = calc_recall(
        indices,
        list(dict.fromkeys(interacted_products_test)) # odstrániť duplikáty z pola produktov pozretých v testovacej sade
    )
    
    # 9. vrátenie výsledkovi
    return distances, indices, precision, recall

# Metriky

In [84]:
# metriky (okrem prvých dvoch) prevzaté z https://gist.github.com/bwhite/3726239

def calc_precision(predicted_item_ids, future_touched_item_ids):
    tp = list(intersection(predicted_item_ids, future_touched_item_ids))
    return (len(tp) / min(len(predicted_item_ids), len(future_touched_item_ids)))


def calc_recall(predicted_item_ids, future_touched_item_ids):
    tp = list(intersection(predicted_item_ids, future_touched_item_ids))
    return len(tp) / len(future_touched_item_ids)


def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2))  

In [95]:
# všetci customeri z trénovacej množiny
customers_train = list(dict.fromkeys(df_train['customer_id'].values))
# všetci customeri z testovacej množiny
customers_test = list(dict.fromkeys(df_test['customer_id'].values))
# prienik test a train customerov
customers = intersection(customers_train, customers_test)

evaluate_for = 50000

# Customeri s vysokou presnostou
# 1798323 80% 
# 1285013 70% 

In [96]:
# funkcia na rozdelenie poľa do n rovnakých polí
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
        
# náhodne pomiešaj customerov
#random.shuffle(customers)
        
# rozdel zákazníkov na časti rovnakej dĺžky (n)
chunks_customers = chunks(customers[:evaluate_for], 5000)
chunks_customers = list(chunks_customers)

In [97]:
# zober náhodných n
# customers = random.sample(customers, evaluate_for)


print(f'Total number of customers: {len(customers)}')
print(f'Evaluating for first {evaluate_for} customers.')

average_precisions = []
for customer_group in chunks_customers:
    sum_precision = 0
    sum_recall = 0
    precisions = []
    for idx, customer_id in enumerate(customer_group):
        distances, indices, precision, recall = recommend(model_knn, customer_id, 10)

        sum_precision = sum_precision + precision
        sum_recall = sum_recall + recall

        precisions.append(precision)

        # Výpis precision a recall pre každého zákazníka čo má precision > 0
        if precision > 0:
            print(f'Cust. {idx + 1}.\t{customers[idx]}\t{round(precision * 100, 2)}%\t{round(recall * 100, 2)}%')

    print('\n')
    
    average_precision = sum_precision / evaluate_for
    average_precisions.append(average_precision)
    
    print(f'Overall precision is {average_precision}')
    print(f'Overall recall is {sum_recall / evaluate_for}')

    print('\n')

Total number of customers: 177177
Evaluating for first 50000 customers.
Cust. 21.	1572895	20.0%	20.0%
Cust. 23.	2097187	33.33%	33.33%
Cust. 27.	524332	25.0%	25.0%
Cust. 35.	2097219	100.0%	100.0%
Cust. 39.	1048649	10.0%	6.67%
Cust. 48.	524387	50.0%	50.0%
Cust. 57.	524400	100.0%	100.0%
Cust. 71.	1048722	100.0%	100.0%
Cust. 76.	2097304	28.57%	28.57%
Cust. 95.	180	16.67%	16.67%
Cust. 97.	1573047	25.0%	25.0%
Cust. 98.	1573054	66.67%	66.67%
Cust. 108.	524511	20.0%	20.0%
Cust. 110.	233	14.29%	14.29%
Cust. 124.	271	25.0%	25.0%
Cust. 136.	297	14.29%	14.29%
Cust. 143.	1048890	40.0%	30.77%
Cust. 157.	1573208	33.33%	33.33%
Cust. 162.	2097505	66.67%	66.67%
Cust. 169.	524653	20.0%	20.0%
Cust. 171.	1573231	100.0%	100.0%
Cust. 172.	2097521	20.0%	20.0%
Cust. 183.	1573260	50.0%	50.0%
Cust. 191.	524708	20.0%	5.71%
Cust. 193.	2097574	66.67%	66.67%
Cust. 218.	2097646	20.0%	10.0%
Cust. 219.	497	50.0%	50.0%
Cust. 220.	1573363	50.0%	50.0%
Cust. 226.	510	16.67%	16.67%
Cust. 230.	524813	10.0%	1.39%
Cust. 236.	1

Cust. 1711.	2101338	40.0%	40.0%
Cust. 1715.	2101345	100.0%	100.0%
Cust. 1727.	2101369	100.0%	100.0%
Cust. 1733.	1052807	50.0%	50.0%
Cust. 1755.	2101430	10.0%	9.09%
Cust. 1767.	4305	14.29%	14.29%
Cust. 1771.	528595	100.0%	100.0%
Cust. 1775.	1052893	16.67%	16.67%
Cust. 1777.	1052898	14.29%	14.29%
Cust. 1795.	1052956	14.29%	14.29%
Cust. 1799.	4386	100.0%	100.0%
Cust. 1809.	2101566	100.0%	100.0%
Cust. 1813.	528710	100.0%	100.0%
Cust. 1819.	4434	10.0%	6.25%
Cust. 1825.	1053024	20.0%	6.9%
Cust. 1827.	4458	37.5%	37.5%
Cust. 1828.	1053037	10.0%	5.26%
Cust. 1848.	1577392	33.33%	33.33%
Cust. 1860.	2101704	50.0%	50.0%
Cust. 1863.	1053135	25.0%	25.0%
Cust. 1866.	528851	50.0%	50.0%
Cust. 1868.	1053143	10.0%	8.33%
Cust. 1869.	1053145	33.33%	33.33%
Cust. 1877.	528882	50.0%	50.0%
Cust. 1878.	4596	100.0%	100.0%
Cust. 1882.	1053187	10.0%	5.56%
Cust. 1891.	4629	50.0%	50.0%
Cust. 1897.	2101815	20.0%	20.0%
Cust. 1901.	1053251	40.0%	40.0%
Cust. 1906.	4696	22.22%	22.22%
Cust. 1922.	1577601	20.0%	16.67%
Cust.

Cust. 3254.	8123	100.0%	100.0%
Cust. 3272.	8161	66.67%	66.67%
Cust. 3274.	532449	42.86%	42.86%
Cust. 3279.	1581035	33.33%	33.33%
Cust. 3288.	2105355	10.0%	9.09%
Cust. 3291.	2105360	14.29%	14.29%
Cust. 3312.	532557	100.0%	100.0%
Cust. 3319.	1581146	100.0%	100.0%
Cust. 3321.	1581151	30.0%	25.0%
Cust. 3322.	2105442	100.0%	100.0%
Cust. 3326.	532588	100.0%	100.0%
Cust. 3331.	1056888	50.0%	50.0%
Cust. 3341.	1056904	50.0%	50.0%
Cust. 3351.	1581216	50.0%	50.0%
Cust. 3362.	1056962	16.67%	16.67%
Cust. 3364.	8391	50.0%	50.0%
Cust. 3365.	2105545	50.0%	50.0%
Cust. 3367.	2105548	66.67%	66.67%
Cust. 3378.	1057015	33.33%	33.33%
Cust. 3380.	1581311	50.0%	50.0%
Cust. 3382.	1581321	10.0%	2.27%
Cust. 3384.	2105612	50.0%	50.0%
Cust. 3385.	1581323	25.0%	25.0%
Cust. 3392.	8478	50.0%	50.0%
Cust. 3400.	1581356	22.22%	22.22%
Cust. 3414.	2105681	66.67%	66.67%
Cust. 3435.	532871	100.0%	100.0%
Cust. 3443.	1581471	100.0%	100.0%
Cust. 3467.	2105815	16.67%	16.67%
Cust. 3476.	2105837	50.0%	50.0%
Cust. 3485.	1057281	10

Cust. 4885.	1585098	10.0%	1.06%
Cust. 4907.	1585133	30.0%	15.0%
Cust. 4920.	1585161	22.22%	22.22%
Cust. 4921.	1585163	100.0%	100.0%
Cust. 4941.	12344	11.11%	11.11%
Cust. 4942.	12349	100.0%	100.0%
Cust. 4943.	12351	25.0%	25.0%
Cust. 4944.	536641	100.0%	100.0%
Cust. 4948.	12357	66.67%	66.67%
Cust. 4952.	536644	14.29%	14.29%
Cust. 4957.	1585239	100.0%	100.0%
Cust. 4959.	1060958	66.67%	66.67%
Cust. 4963.	536693	100.0%	100.0%
Cust. 4966.	12423	50.0%	50.0%
Cust. 4967.	12425	100.0%	100.0%
Cust. 4969.	1061002	57.14%	57.14%
Cust. 4970.	2109580	10.0%	1.54%
Cust. 4997.	2109655	50.0%	50.0%


Overall precision is 0.008131515873015875
Overall recall is 0.00788303851213869


Cust. 5.	1572875	10.0%	3.57%
Cust. 8.	524300	50.0%	50.0%
Cust. 28.	2097199	100.0%	100.0%
Cust. 31.	524342	100.0%	100.0%
Cust. 40.	1048652	50.0%	50.0%
Cust. 41.	1048653	20.0%	20.0%
Cust. 43.	85	10.0%	3.33%
Cust. 51.	1048681	100.0%	100.0%
Cust. 62.	129	100.0%	100.0%
Cust. 63.	131	100.0%	100.0%
Cust. 70.	1048721	100.0%	100.0%
Cust. 

Cust. 1477.	527877	10.0%	7.14%
Cust. 1478.	2100742	12.5%	12.5%
Cust. 1480.	527880	100.0%	100.0%
Cust. 1501.	1576502	25.0%	25.0%
Cust. 1503.	527934	20.0%	18.18%
Cust. 1510.	1052244	16.67%	16.67%
Cust. 1511.	1052245	10.0%	8.33%
Cust. 1513.	527956	100.0%	100.0%
Cust. 1523.	2100849	33.33%	33.33%
Cust. 1529.	1576581	100.0%	100.0%
Cust. 1532.	2100875	10.0%	5.26%
Cust. 1536.	528021	100.0%	100.0%
Cust. 1540.	2100897	30.0%	14.29%
Cust. 1551.	1052343	33.33%	33.33%
Cust. 1553.	2100922	50.0%	50.0%
Cust. 1554.	2100929	66.67%	66.67%
Cust. 1567.	3818	50.0%	50.0%
Cust. 1569.	1052402	10.0%	4.55%
Cust. 1578.	1052429	50.0%	50.0%
Cust. 1590.	2101048	33.33%	33.33%
Cust. 1594.	528197	100.0%	100.0%
Cust. 1597.	1052491	100.0%	100.0%
Cust. 1598.	2101066	10.0%	7.14%
Cust. 1600.	2101081	33.33%	33.33%
Cust. 1601.	1052507	20.0%	20.0%
Cust. 1605.	528232	66.67%	66.67%
Cust. 1613.	528240	20.0%	11.11%
Cust. 1619.	1052546	50.0%	50.0%
Cust. 1620.	1052548	10.0%	10.0%
Cust. 1631.	2101155	16.67%	16.67%
Cust. 1644.	4031	100

...

Cust. 3472.	1581541	10.0%	9.09%
Cust. 3474.	2105833	75.0%	75.0%
Cust. 3475.	8683	100.0%	100.0%
Cust. 3477.	1581550	33.33%	33.33%
Cust. 3484.	532991	30.0%	15.79%
Cust. 3489.	8716	10.0%	10.0%
Cust. 3490.	1581582	14.29%	14.29%
Cust. 3492.	2105878	20.0%	20.0%
Cust. 3500.	8738	100.0%	100.0%
Cust. 3502.	8746	100.0%	100.0%
Cust. 3505.	8759	10.0%	6.67%
Cust. 3515.	8792	28.57%	28.57%
Cust. 3540.	2106008	10.0%	7.69%
Cust. 3543.	8868	33.33%	33.33%
Cust. 3549.	533170	20.0%	20.0%
Cust. 3555.	1057470	100.0%	100.0%
Cust. 3557.	1581763	20.0%	20.0%
Cust. 3579.	2106111	50.0%	50.0%
Cust. 3581.	1581824	10.0%	3.12%
Cust. 3593.	533280	40.0%	40.0%
Cust. 3598.	533291	100.0%	100.0%
Cust. 3600.	1057584	100.0%	100.0%
Cust. 3606.	2106171	10.0%	10.0%
Cust. 3612.	2106188	50.0%	50.0%
Cust. 3615.	533330	33.33%	33.33%
Cust. 3616.	9043	20.0%	20.0%
Cust. 3625.	9066	100.0%	100.0%
Cust. 3638.	9111	10.0%	2.63%
Cust. 3649.	1581998	10.0%	4.76%
Cust. 3656.	2106300	100.0%	100.0%
Cust. 3662.	2106313	50.0%	50.0%
Cust. 3664.	5334

Cust. 55.	2097263	100.0%	100.0%
Cust. 59.	524412	33.33%	33.33%
Cust. 61.	1048705	40.0%	40.0%
Cust. 62.	129	50.0%	26.32%
Cust. 66.	1573002	66.67%	66.67%
Cust. 70.	1048721	33.33%	33.33%
Cust. 84.	2097319	100.0%	100.0%
Cust. 89.	2097324	50.0%	50.0%
Cust. 95.	180	25.0%	25.0%
Cust. 111.	1048818	25.0%	25.0%
Cust. 115.	524541	40.0%	40.0%
Cust. 121.	524553	12.5%	12.5%
Cust. 122.	1573133	25.0%	25.0%
Cust. 128.	1573141	66.67%	66.67%
Cust. 131.	1048862	100.0%	100.0%
Cust. 142.	1048888	10.0%	5.56%
Cust. 146.	2097473	33.33%	33.33%
Cust. 155.	524623	100.0%	100.0%
Cust. 157.	1573208	50.0%	50.0%
Cust. 175.	1048956	10.0%	4.0%
Cust. 176.	382	100.0%	100.0%
Cust. 188.	405	33.33%	33.33%
Cust. 208.	2097612	66.67%	66.67%
Cust. 227.	1573374	40.0%	36.36%
Cust. 238.	524834	10.0%	7.69%
Cust. 251.	1573445	12.5%	12.5%
Cust. 252.	2097735	10.0%	10.0%
Cust. 253.	585	100.0%	100.0%
Cust. 258.	524880	30.0%	23.08%
Cust. 275.	654	100.0%	100.0%
Cust. 276.	656	90.0%	75.0%
Cust. 279.	662	25.0%	25.0%
Cust. 280.	1573525	40.0%	

Cust. 1783.	1577211	100.0%	100.0%
Cust. 1785.	528637	20.0%	20.0%
Cust. 1787.	528651	50.0%	50.0%
Cust. 1788.	2101521	100.0%	100.0%
Cust. 1790.	4369	50.0%	50.0%
Cust. 1794.	1577242	60.0%	60.0%
Cust. 1800.	2101539	50.0%	50.0%
Cust. 1802.	1577256	33.33%	33.33%
Cust. 1806.	1052980	22.22%	22.22%
Cust. 1811.	528703	100.0%	100.0%
Cust. 1820.	1577300	100.0%	100.0%
Cust. 1825.	1053024	10.0%	1.54%
Cust. 1826.	1577311	100.0%	100.0%
Cust. 1834.	1577346	25.0%	25.0%
Cust. 1835.	2101636	25.0%	25.0%
Cust. 1836.	4486	10.0%	6.67%
Cust. 1851.	1053110	20.0%	8.0%
Cust. 1853.	1053112	12.5%	12.5%
Cust. 1860.	2101704	66.67%	66.67%
Cust. 1861.	528845	40.0%	40.0%
Cust. 1866.	528851	40.0%	40.0%
Cust. 1888.	1053198	50.0%	50.0%
Cust. 1891.	4629	20.0%	10.53%
Cust. 1899.	1053247	33.33%	33.33%
Cust. 1901.	1053251	100.0%	100.0%
Cust. 1914.	1053299	100.0%	100.0%
Cust. 1916.	2101880	10.0%	5.0%
Cust. 1919.	1053308	20.0%	20.0%
Cust. 1921.	4736	12.5%	12.5%
Cust. 1923.	1053314	14.29%	14.29%
Cust. 1924.	4739	14.29%	14.29%
Cus

Cust. 3358.	1056947	66.67%	66.67%
Cust. 3379.	1581308	100.0%	100.0%
Cust. 3387.	1581327	28.57%	28.57%
Cust. 3390.	1581333	100.0%	100.0%
Cust. 3395.	1581350	50.0%	50.0%
Cust. 3397.	1057065	100.0%	100.0%
Cust. 3399.	1057067	50.0%	50.0%
Cust. 3401.	532782	14.29%	14.29%
Cust. 3405.	2105653	33.33%	33.33%
Cust. 3408.	2105666	16.67%	16.67%
Cust. 3425.	1057127	10.0%	8.33%
Cust. 3433.	532862	100.0%	100.0%
Cust. 3436.	8584	20.0%	13.33%
Cust. 3445.	8612	20.0%	20.0%
Cust. 3449.	2105776	50.0%	50.0%
Cust. 3458.	1057220	37.5%	37.5%
Cust. 3467.	2105815	16.67%	16.67%
Cust. 3471.	8677	20.0%	10.53%
Cust. 3478.	532975	10.0%	7.14%
Cust. 3481.	532982	75.0%	75.0%
Cust. 3482.	1581565	14.29%	14.29%
Cust. 3483.	532990	20.0%	10.0%
Cust. 3493.	1581590	10.0%	4.76%
Cust. 3506.	2105912	40.0%	40.0%
Cust. 3510.	1581635	100.0%	100.0%
Cust. 3517.	533088	33.33%	33.33%
Cust. 3521.	1581669	28.57%	28.57%
Cust. 3523.	1581671	33.33%	33.33%
Cust. 3527.	2105977	10.0%	6.25%
Cust. 3534.	2106000	75.0%	75.0%
Cust. 3547.	2106032	100

Cust. 4923.	536589	10.0%	6.67%
Cust. 4924.	2109455	100.0%	100.0%
Cust. 4927.	2109462	100.0%	100.0%
Cust. 4934.	536616	10.0%	5.26%
Cust. 4948.	12357	100.0%	100.0%
Cust. 4952.	536644	100.0%	100.0%
Cust. 4965.	1060998	100.0%	100.0%
Cust. 4972.	536713	12.5%	12.5%
Cust. 4979.	12459	16.67%	16.67%
Cust. 4986.	2109618	50.0%	20.83%
Cust. 4989.	536765	33.33%	33.33%
Cust. 4993.	1585360	100.0%	100.0%


Overall precision is 0.007818626984126989
Overall recall is 0.007543433285240862




In [103]:
# v dátach je chyba, delil som ich číslom 50 000 a nie 5 000 (vzorka bola po 5 000)
average_precisions = list(map(lambda x: x * 10, average_precisions))
np.mean(average_precisions)

0.802123888888889